# Data Engineer's basic tasks

Here, I have completed several basic steps of the everyday Data Engineer's life and provided comments on my own thoughts regarding the execution methods, my logic, and the libraries I've chosen. I enjoyed working on this task and made every effort to explain my way of thinking. I hope we can find a connection. In any case, I am ready to answer all your questions and provide a more detailed explanation of my solution.

**So, let's get started!**

## 1. Getting data from an API

I divided the execution of this task into two functions for logical reasons. The first function ___get_data_from_source___ connects to the API and retrieve data. The second function ___store_data___ saves the obtained data into a JSON file, as specified in the task.

In [ ]:
import requests

def get_data_from_source(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data

    except requests.exceptions.HTTPError as http_err:
        raise http_err
        return None

This is how the function looks when connecting to a real API.   
- I used a ___try-except___ block to prevent the code from terminating in case of an error and to capture the error for analysis and code improvement, depending on the specific behavior of the API.  
- The line ___'response.raise_for_status()'___ precisely checks the server response status and captures the 'requests.exceptions.HTTPError' exception, which contains information about the error code and message.  
- I used ___requests___ library because it provides a simple and convenient way to make HTTP requests to web resources.

Since we don't have a working server in this specific case, I simply read data from an existing JSON file and will write it to a separate JSON file.

In [ ]:
import json

def get_data_from_source(filename):
    try:
        with open(filename, 'r') as input_file:
            data = json.load(input_file)
        return data

    except FileNotFoundError:
        print(f'File not found: {filename}')
        return None

    except json.JSONDecodeError as e:
        print(f'JSON decoding error: {str(e)}')
        return None

    except Exception as e:
        print(f'FAIL while get_data_from_source: {str(e)}')
        return None

- Here, I used a ___"with"___ of context manager to work with the file, which automatically closes the file and releases memory when the function execution is completed.  
- The parameter ___'r'___ signifies that we are opening the file for reading only.  
- Additionally, I have handled certain ___exceptions___ that may occur if the file is not found or if the data in it doesn't conform to the JSON format.
- Also ___json___ library allows working with JSON files as python objects.

I used arguments in both functions to allow them work with different sourses, not only with specified ones.

In [ ]:
def store_data(data):
    try:
        with open('data.json', 'a') as outfile:
            json.dump(data, outfile, indent=4)
    except Exception as e:
        print(f"FAIL while store_data: {str(e)}")

This function takes data and saves it to a separate file.  

- I used the ___'a'___ argument to append data to the file with each function call, preserving the previous data. If we need to overwrite the data in the file, we can use the ___'w'___ parameter. The file will also be created if it does not exist beforehand.
- Part of ___'indent=4'___ helps make the data prettier and more readable by adding 4 spaces.
- To track the progress of the program, I use regular ___prints___, but for more high-quality, convenient, and reliable tracking, ___logging___ can be used.

## 2. Data clean-up

Data cleaning is the process of identifying and correcting or removing errors, inconsistencies, and inaccuracies in data sets. It involves checking data for completeness, removing duplicate entries, dealing with missing data, standardizing data formats, and correcting data values that are out of range or invalid.

- Accurate data is essential for making informed decisions: If the data is incorrect, any insights or conclusions drawn from it may be flawed.
- Data cleaning can help identify and prevent errors early on: This can save time and resources by avoiding costly mistakes downstream.
- Data cleaning can improve the quality of data: By removing errors and inconsistencies, data becomes more reliable and trustworthy.

##### In this specific case, we have a small amount of data, so data cleanliness analysis can be easily done manually.

At first glance, the __'Quantity (£)'__ column caught my eye for two reasons - it has the ___currency name___ in the values and also contains ___negative___ values. Clearly, for financial analysis, the data type should be numeric, and the commonly used data type for this purpose is 'float' with two decimal places. Therefore, we should _remove_ the currency mention and _convert_ type.  
As for the negative values, they deserve special attention. Initially, we might consider simply deleting this row because in the context of electricity consumption, negative values imply 'energy generation,' and we assume that the Glasgow office is not involved in energy production. However, upon closer examination, I notice that it pertains to only one day, unlike the other data, which represents reports for specific months. Furthermore, this row shares a similar account number with the one above it, where the office name and location match, but with the addition of _'-re'_, which could indicate a debt or a case where the company initially reserved more electricity than it used, for instance. So, I would definitely _keep_ this row, although it could also be logically _combined_ with the one indicating the period to which it belongs. I would discuss this aspect with a colleague as the choice depends on the goals of further data analysis.    
I also noticed the outlier values because the value of 5000 is significantly different from the average. I would discuss this with a more experienced colleague, but I personally find it quite logical that energy consumption and its cost increase in November as the daylight hours and temperature decrease compared to October. Although the values in November are twice as high as in December, this can be explained by the greater number of working days in the office in November compared to December and its calendar holidays. And even when comparing this value to the value for the same period but for a different office (Spain), it can be explained by geographical weather data and a significant _difference in the number of office employees_.

Additionally, I observe that __'Bill number'__ names follow different naming conventions for different offices. In my current solution, I did not modify this column as I believe that the naming logic is entirely unique to each office location and does not repeat. However, this can certainly be discussed with a colleague, as standardized bill _numbers_ might be necessary for analysis.

I also noticed variations in categorical names in the __'Country'__ column for the same location. Therefore, in my solution, I _replace_ 'United Kingdom' with 'GB'.

The values in __'Time Period'__ column follow the same logic, but to work with dates, it need to _be converted_ to a data type appropriate for dates.

The __'Supplier'__ and __'Tariff Name'__ columns _do not raise any questions_ for me.

However, regarding the values in the __'Price per kWh'__ column, I would make some changes - it's necessary to _remove_ the currency mention, _convert_ the column to a numeric 'float' data type, and for convenience, _convert_ 'p' to 'GBP' to have consistent numerical scales for each financial column.

##### However, in a real-world scenario with a large volume of data, we should automate the process to speed up work and save human resources.
The initial steps involve reading data from a file and displaying a few data rows for human inspection.

(___Pandas___ library is a perfect fit for this task as it allows easy data reading from various sources (in our case, a CSV file) and offers a plethora of methods for filtering, replacing, transforming, and removing data.)

In [1]:
import pandas as pd

df = pd.read_csv('utilities.csv', delimiter=',')

In [2]:
df.head(10) #to print first 10 rows

,Facility Name,Bill number,Quantity (£),Location,Country,Time Period,Supplier,Tariff Name,Price per kWh
0,HQ,hq-001,1000,Headquarters,United Kingdom,Oct-1 - Oct-31 2021,EnergyCorp,Renewable 100,25p
1,HQ,hq-002,5000,Headquarters,United Kingdom,Nov-1 - Nov-30 2021,EnergyCorp,Renewable 100,25p
2,HQ,hq-003,2500,Headquatrers,United Kingdom,Dec-01 - Dec-31 2021,EnergyCorp,Renewable 100,25p
3,Office Scotland,1235,900,Glasgow,GB,Nov-1 - Nov-30 2021,EnergyCorp,coal-only,16p
4,Office Scotland,0815,1000,Glasgow,GB,Dec-1 - Dec-31 2021,PowerSupply,PowerSupply Budget,18p
5,Office Scotland,0815-re,-200,Glasgow,GB,Dec-29 - Dec-29 2021,PowerSupply,PowerSupply Budget,18p
6,Office Spain,December,1000 GBP,Spain Office,ESP,Nov-1 - Nov-30 2021,EnergyCorp,coal-only,16p


At this stage, in addition to the issues I've already described above, I've also noticed extra spaces that need to be removed:

In [3]:
df = df.map(lambda x: x.rstrip())

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Facility Name  7 non-null      object
 1   Bill number    7 non-null      object
 2   Quantity (£)   7 non-null      object
 3   Location       7 non-null      object
 4   Country        7 non-null      object
 5   Time Period    7 non-null      object
 6   Supplier       7 non-null      object
 7   Tariff Name    7 non-null      object
 8   Price per kWh  7 non-null      object
dtypes: object(9)
memory usage: 632.0+ bytes


Analyzing this output helps determine which columns contain non-null values and their data types. Here, we can see that all columns in our data are filled, which meets data cleanliness standards. We also observe the data types; all columns have the 'object' data type, typically used for textual data. However, considering the context, columns like 'Quantity (£)' and 'Price per kWh' should be converted to 'float', while 'Time Period' should be converted to 'date'.

Let's start with the 'Quantity (£)' column.   
I use a function that converts the column to a numerical float type. I specify 'errors='raise'' to catch any errors that might occur. (Assuming I haven't seen the complete data, and there don't appear to be any obvious issues with the column in the initial sample.)

In [12]:
df['Quantity (£)'] = pd.to_numeric(df['Quantity (£)'], errors='raise')

We see that the error arises from the sixth row, where the value is '1000 GBP.' Let's remove this currency mention from all rows that 'might still exist' in our data using simple regular expression.

In [6]:
df['Quantity (£)'] = df['Quantity (£)'].str.replace(' GBP', '', regex=True)

Now our data can be converted to another data type again. If I encountered another error, for example, another currency mention, I would handle it similarly or take steps based on the specific situation. We follow a similar approach with the column 'Price per kWh'.

In [7]:
df['Price per kWh'] = df['Price per kWh'].str.replace('p', '', regex=True)

In [8]:
df['Price per kWh'] = pd.to_numeric(df['Price per kWh'], errors='raise')

Additionally, as part of working with this column, I convert pennies to British pounds.

In [9]:
df['Price per kWh'] = df['Price per kWh'] / 100

When it comes to the 'Time Period', I split a single column into two - one for the start date and the other for the end date of the period. Since our source data has a complex structure, I use regular expressions for precise date extraction, involving three letters for the month, one or two digits for the day, and four digits for the year. The extracted data is distributed into auxiliary columns, which are later removed. Then, I use the ___'pd.to_datetime'___ method to convert the format into the expected date format.

In [10]:
date_pattern = r'(\w{3}-\d{1,2}) - (\w{3}-\d{1,2}) (\d{4})'
df[['Start', 'End', 'Year']] = df['Time Period'].str.extract(date_pattern)

df['Start Date'] = pd.to_datetime(df['Start'] + ' ' + df['Year'], format='%b-%d %Y')
df['End Date'] = pd.to_datetime(df['End'] + ' ' + df['Year'], format='%b-%d %Y')

df = df.drop(columns=['Time Period', 'Start', 'End', 'Year'])

We it is needed to check the boundary values. Let's do this using the 'Quantity (£)' column as an example.

In [13]:
print(df['Quantity (£)'].max())
print(df['Quantity (£)'].min())

5000
-200


We've already discussed boundary values earlier. Now, let's check categorical values. I do this using the 'Country' column as an example.

In [14]:
df['Country'].value_counts()

Country
United Kingdom    3
GB                3
ESP               1
Name: count, dtype: int64

I standardize different variations of names for the same semantic category. Since it is used an abbreviation for The Kingdom of Spain, I also choose 'GB' for The United Kingdom of Great Britain and Northern Ireland.

In [15]:
df['Country'] = df['Country'].str.replace('United Kingdom', 'GB', regex=True)

Also, if we check 'Location' column, we notice misspelling:

In [18]:
df['Location'].value_counts()

Location
Headquarters    3
Glasgow         3
Spain Office    1
Name: count, dtype: int64

In [17]:
df['Location'] = df['Location'].str.replace('Headquatrers', 'Headquarters', regex=True)

Now it's time to save the cleaned data into a new file without including the index column.

In [19]:
df.to_csv('cleaned_utilities.csv', index=False)

## 3. Data wrangling

Considering both the columns from the specification and the clean date columns I obtained in the previous step, I would design the data as follows:

![db_design](db_design.png)

I attempted to logically combine certain columns and isolate others that are independent of each other. All IDs in my database design are of integer type and automatically increment when new rows are added. 'Tariff' is a renamed column from 'Price per kWh.' To calculate the 'Spend' column, I divided 'Quantity' by 'Tariff'.

Here is the pandas code for turning data into specification format.

In [20]:
df = pd.read_csv('cleaned_utilities.csv')

Rename columns:

In [21]:
df.rename(columns={'Facility Name': 'Facility_name',
                   'Bill number': 'Bill_name',
                   'Quantity (£)': 'Quantity',
                   'Location': 'Location',
                   'Country': 'Country',
                   'Supplier': 'Supplier',
                   'Tariff Name': 'Tariff_name',
                   'Price per kWh': 'Tariff',
                   'Start Date': 'Start_date',
                   'End Date': 'End_date'}, inplace=True)

Add new necessary columns:

In [22]:
df['Currency'] = 'GBP'
df['Unit'] = 'kWh'
df['Spend'] = df['Quantity'] / df['Tariff']
df['Spend'] = df['Spend'].round(2)  #round to second decimal

df['Facility_ID'] = df.groupby('Facility_name').ngroup() + 1
df['Utility_ID'] = df.groupby('Bill_name').ngroup() + 1

Create result file for specification:

In [19]:
columns_to_export = ['Facility_ID', 'Utility_ID', 'Tariff', 'Quantity', 'Unit', 'Spend', 'Currency', 'Start_date', 'End_date']
result_df = df[columns_to_export]
result_df.to_csv('output_report.csv', index=False)

Take a glance on the results.

In [26]:
result_df

,Facility_ID,Utility_ID,Tariff,Quantity,Unit,Spend,Currency,Start_date,End_date
0,1,5,0.25,1000,kWh,4000.00,GBP,2021-10-01,2021-10-31
1,1,6,0.25,5000,kWh,20000.00,GBP,2021-11-01,2021-11-30
2,1,7,0.25,2500,kWh,10000.00,GBP,2021-12-01,2021-12-31
3,2,3,0.16,900,kWh,5625.00,GBP,2021-11-01,2021-11-30
4,2,1,0.18,1000,kWh,5555.56,GBP,2021-12-01,2021-12-31
5,2,2,0.18,-200,kWh,-1111.11,GBP,2021-12-29,2021-12-29
6,3,4,0.16,1000,kWh,6250.00,GBP,2021-11-01,2021-11-30


When I generated the resulting data, I had some questions about the 'Spend' column, where I calculated energy expenses. The values seemed controversial to me. Therefore, I would like to consult with a colleague regarding my decision.    
Additionally, there is another scenario in which the 'Quantity (£)' column was initially named incorrectly and represented not the cost of electricity but its consumption in kWh. In that case, a different approach would be needed for the value '1000 GBP'. And 'Spend' column would be the result of multiplying the 'Tariff' by 'Quantity' columns.